# Conda - or how to handle too many monkey fighting snakes on this Monday-to-Friday plane!

## Why do I need a virtual environment?

In principle, you can use the python version shipped with the system.

Problems:

- you have root rights and want to install a package which breaks the system
- you do not have root rights and want to install a newer version or a special package
- different projects need different versions of python/packages

## 1. Root rights

`pip install numpy`

Problems:

- pip does not have true dependency resolution
- pip could easily install a version or versions of dependencies which break your system
- if the system is updated, your code can brake as packages may conflict
- use system package manager

## 2. No root rights

`pip install --user 'numpy>=1.13,<1.18'`

Problems:

- you have to set the path variables correctly (usually ./local)
- if the system is updated, your code can brake as packages may conflict
- only safe if you install your own python version

## 3. Working on different projects

Problems:

- you want to use code written for older python/package versions
- you are working on several projects requiring different versions of a package
- you want to share code with people who have different python/package versions installed


## Solution: Virtual environments

- directory with isolated, self-contained python installation
- allows absolute control over packages and versions
- easy switching between different environments
- easy exporting/importing/sharing environments

A virtual environment creates an isolated python executable.

Once activated it sets the environment folder to the beginning of `$PATH`.

The activate virtual environment is indicated in the command line as:

`(environmentname) user@machine`

If python is executed then it looks for packages in the local folder.

## Option A: virtualenv

https://virtualenv.pypa.io

To install virtualenv use

```pip install virtualenv```

Create a folder with a new environment using the according python interpreter

```virtualenv <dirname>```

However, the python interpreter can be specified with

```virtualenv -p <absolute-path-to-python> <dirname>```

Virtualenv can use the packages which are already installed and will symlink them with the option

```virtualenv --system-site-packages <dirname>```

To activate a virtualenv environment execute

```source <dirname>\bin\activate```

Additional packages are installed via pip into the new environment folder.

To deactivate the environment just execute

```deactivate```

You can export the environment packages with

`pip freeze > requirements.txt`

and then import them on in a different environment/machine with

`pip install -r path/to/requirements.txt`

Python has this functionality built in since 3.3, first as pyvenv which was replaced by venv after 3.6.

https://docs.python.org/3/library/venv.html

Advantages of venv:
- already shipped with python
- does not need to hard copy python executable

Disadvantages:
- cannot create environments for arbitrarily installed python versions

## Option B: pipenv

https://pypi.org/project/pipenv/

pipenv was developed in order to have a streamlined and improved version of virtualenv but has not been updated since 2018!

Install it via

`pip install pipenv`

Create an environment in your project folder 

`pipenv install`

This creates a `Pipfile`, a `Pipfile.lock` and an environment stored in `.local`

Activate your environment with

`pipenv shell`

### 1. Combines the functionality of pip and virtualenv

new environments are created via

`pipenv install`

packages are installed via

`pipenv install <package>`

### 2. Environments are stored in a central place

Your system is not cluttered by environment folders which are forgotten over time but are all stored in

`~/.local/share/virtualenvs/<hashed name>`

### 3. Uses Pipfile for dependecy resolution

`pip freeze > requirements.txt`

stores strict version numbers of packages and dependencies in plain text file. However, often only a range of versions of a dependency is needed.

Pipfile serves as a way to share used packages, resolves versions and can check for security updates of packages.

- Pipfile is automatically updated as soon as you install new packages. You can also use the `--dev` flag for packages you only need for development
- Pipfile.lock is used for sharing a version definite version of the Pipfile and can be updated via `pipenv lock`

Creating an environment in a folder automatically locates existing Pipfiles and installs the necessary packages.

## Option C: conda

https://docs.conda.io/en/latest/index.html

Conda has additional benefits to pipenv:

- cross platform
- language agnostic
- access to additional packages from Anaconda repositories

Installation is done via a script from the official website.

Conda was developed as part of the Anaconda distribution of python/R

https://www.anaconda.com/products/individual
    
which is a 1500 package heavy 5gb installation.

Conda is now developed separately and can be installed without any packages as miniconda

https://docs.conda.io/en/latest/miniconda.html

Conda looks for packages in the default Anaconda channels (compared to PyPI for pip). But additional channels can be added (e.g. [conda-forge](https://conda-forge.org/)).

Anaconda also comes with a GUI, Anaconda navigator which also can be installed via conda

![](https://www.anaconda.com/imager/assets/Products/Product-Screenshots/5736/navigator-screenshot_680db6b6f11f9cc710dd7defae241cd3.png)

## Creating environments

To create an empty environment use

`conda create --name myenv` or `conda create -n myenv`

Or directly create a minimal python environment via

`conda create --name myenv python`

which uses the python version installed in the base environment.

You can also specify the version with

`conda create --name myenv python=3.5`

List all installed environments

`conda env list`

Remove an environment with

`conda env remove --name myenv`

The standard save location is within your miniconda installation

`~/miniconda/envs/` or `~/miniconda3/envs/`
 
However, you can install it to any directory with
 
`conda create --prefix /path/to/venv` or `conda create -p /path/to/venv`

Be aware, to activate special directories you will have to provide the full path which is also displayed in the command line promt. (Latter can be fixed by editing `.condarc`)

## Activating environments

Activate an environment with its name via

`conda activate myenv`

The active environment is listed with an asterisk in `conda env list`.

Deactivate an environment with

`conda deactivate`

Be aware, that the default behavior of conda is to allow access to all global packages. You can completely isolate your environment by using

`export PYTHONNOUSERSITE=True`

before activation.

## Installing packages



Once activated, you can now install new packages or specific versions into the environment via

`conda install scipy` or `conda install scipy=0.15.0`

List all installed packages with

`conda list`

and remove packages with

`conda remove scipy`

You can search for packages in Anaconda or other channels

`conda search --channel conda-forge <packagename>`

Conda also installs pip in every python environment and also is aware of pip installed packages (the channel is indicated by `conda list`). But do not use the `--user` argument and be aware that pip can brake your environment!

Conda also keeps a history of your changes which you can list with

`conda list --revisions` or `conda list -rev`

To go back to a specific revision, get the revision number from the list and run

`conda install --rev REVNUM`

## Updating packages

To update all packages to newest possible version use

`conda update --all`

or a specific package with

`conda update python`

Conda will resolve dependency conflicts and usually warns very explicitly if they occur.

Dependencies for a package can be displayed with

`conda info <package_name>`

It is recommended to update/install all packages at once so that all dependencies can be resolved at the same time, e.g.

`conda install scipy curl`

However, you can also force a package to stay at a specific version by adding a file named `pinned` to the `conda-meta` directory with the package name and version.

## Cloning and exporting

You can clone environments via

`conda create --name myclone --clone myenv`

To export all packages and dependencies to a file use

`conda env export > environment.yml`

Recreate the environment by using

`conda env create -f environment.yml`

For cross-platform compatibility it is recommended to use

`conda env export --from-history > environment.yml`

which only exports the main installed packages as dependencies could vary across platforms.

## Bonus: Using a virtual environement as a jupyter notebook kernel

Install `ipykernel` in the virtual environment

`conda install ipykernel`

then run

`python -m ipykernel install --user --name=myenv`

Now jupyter notebook should display the new kernel.

Remove it with

`jupyter kernelspec uninstall myenv`